# Demo Code with Face and Hand Recognition

This code is for running the simple demo (prototype) of the system with face and hand recognition models.  
If you can't run another demo code, this is the easiest way to see a result.  
There are some libraries to run this code on the first cell, so please check it.  
It is recommend to run the code with Jupyter Notebook. Run each cell in order.  

- Recommend IDE: Jupyter Notebook, Visual Sutdio Code
- Language: **Python 3.10**

## 1. Import and Load Models

In [2]:
# Basic libraries
import cv2
import os
import math
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp

# For models
import face_recognition
from sklearn.svm import SVC 
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

In [3]:
# Load models
# Cofiguration: set the specific model name and the root
path = os.getcwd() # current path
print(path)        # for checking 

# For hand recognition model (XGB Classifier)
model_xgb = pickle.load(open(path + '/Model/HAND/1025model_xgb.h5', 'rb'))   # config

# For face recognition model (PCA and SVM)
model_svm = pickle.load(open(path + '/Model/FACE/1207_svm_20064.sav', 'rb')) # config
model_pca = pickle.load(open(path+'/Model/FACE/1207_pca_20064.pkl', 'rb'))   # config
# Import haar cascade classifier from cv2 
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') 

C:\Users\mtang\Desktop\SW\@KSW Gangsture\dolphin


C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\mtang\anaconda3\envs\mp\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator PCA from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own ris

## 2. Define Helper Functions

- detect_face
- calc_relative_coord
- preprocess_facedata
- preprocess_handdata
- calc_bounding_rect
- draw_bounding_rect

In [4]:
##### Two functions for detecting and calculating information #####
# Detect face on the image and return coordinates 
def detect_face(gray_img, face_cascade = face_cascade, width=200, height=200) :
    # Setting detector function 
    coordinate = face_cascade.detectMultiScale( 
            gray_img,          # input grayscale image 
            scaleFactor=1.3,   # imgae pyramid scale 
            minNeighbors=4)    # neighbor object minimum distance pixels 
    
    # Detect face coordinate location in image, i.e, detect face
    # If the face was not detected, 
    if coordinate is None or len(coordinate)==0 : 
        return None
    else : 
        # If faces were detected (two more), 
        if len(coordinate) == 2 :
            x1,y1,w,h = coordinate[-1].squeeze()
        # If just one face was detected,
        else : 
            x1,y1,w,h = coordinate[0]        
        x1, y1 = abs(x1), abs(y1)
        x2 = abs(x1+w)
        y2 = abs(y1+h)
        
    face = gray_img[y1:y2, x1:x2]
    face = cv2.resize(face, (width, height))
    
    return face

# Calculate and Create the relative coordinates from the original one
def calc_relative_coord(landmarks) :
    lms_df = pd.DataFrame(landmarks).drop(0, axis=1)
    relative_list = []
    
    base_lm = lms_df.iloc[0, 1:].values.astype(float)
    for i in range(0, 21):
        target_lm = lms_df.loc[i, 2:].values.astype(float)
        result_lm = target_lm - base_lm
        relative_list.append(result_lm)
    
    relative_df = pd.DataFrame(relative_list)
    new_df = pd.concat([lms_df, relative_df], axis=1)

    return new_df


##### Two functions for preprocessing face and hand data as input data #####
# Preprocess for hand dataset
def preprocess_handdata(data) :
    # store hand_type value
    hand_type = data.iloc[0, 0] 
    
    # remove hand_type value column 
    data = data.iloc[:, 1:].copy()
    
    # converting to numpy array 
    data_np = np.array(data)
    data_np = data_np.flatten()
    
    # insert hand_type value 
    if hand_type == "right" :
        data_np = np.insert(data_np, 0, 0)
    else :
        data_np = np.insert(data_np, 0, 1)
    
    # re-shape for input data format of trained model 
    input_data = data_np.reshape(1, data_np.shape[0])

    return input_data

# Preprocess for face dataset
def preprocess_facedata(data, pca=model_pca) :
    data = data.flatten().astype('int').reshape(1,-1)
    input_data = pca.transform(data)
    
    return input_data


##### Two functions for bounding rectangle #####
# Calculate bounding rectangle
def calc_bounding_rect(image, landmarks):  
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)
    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


# Draw bounding rectangle 
def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)
    return image

## 3. Start Collecting Face Dataset
The simple demo code to test the entire system with web cam  
Originally, this demo will show the face label predicted by recognition machine learning model,  
**but it was changed to be available when the face is recognized for testing smoothly.**   
So if the face is detected, the hand will be recognized. The original codes are also written by comments. Please note these changes. 
- to quit the code, press 'q' 

In [5]:
# Setting for Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,              # Only detect one hand 
                       min_detection_confidence=0.7) # defualt 0.5

In [7]:
# Initialize the webcam 
video_capture = cv2.VideoCapture(0) # 0 is a default embedded camera

# Initialize for dataset
landmarks = []
df = pd.DataFrame()
flag = False # If the flag is False, hand recognition is not working 


while True:
    # Read each frame from the webcam
    _, frame = video_capture.read()
    x, y, c = frame.shape
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # For hand
    framegray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # For face
    
    
    # Detect and extract face from gray frame 
    coords = detect_face(framegray)
    
    # Preprocessing for input face data 
    if coords is not None: 
        flag = True
        # Input data for face recognition
        input_face = preprocess_facedata(coords)
        pred_proba_face = model_svm.predict_proba(input_face)
        # If the probability of detected face was lower than threshold 
#         if np.max(pred_proba_face) <= 0.5 : # config: threshold is 0.5
#             flag = False 
#         # Predict face input
#         pred_face = np.argmax(pred_proba_face)
    # If face is not detected
    else: 
        flag = False 
    
    
    # Check the authority with "flag" status, Can we use the gesture controller? 
    if flag == True :
        # Show the autorithy with face prediction label 
#         cv2.putText(frame, "Accessed! with face label "+str(pred_face), (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frame, "Accessed!", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Mediapipe processing
        result = hands.process(framergb)
        landmarks.clear() # For empty list 
        if result.multi_hand_landmarks:
            for handslms, handness in zip(result.multi_hand_landmarks,  # Be carefule to set attribute for right coordinate system
                                          result.multi_handedness): 
                for point in mp_hands.HandLandmark: # 0 ~ 20
                    x = handslms.landmark[point].x
                    y = handslms.landmark[point].y
                    z = handslms.landmark[point].z
                    landmarks.append([str(point), handness.classification[0].label, x, y, z])

                # Draw landmarks on frames with bounding rectangle
                brect = calc_bounding_rect(frame, handslms)
                frame = draw_bounding_rect(True, frame, brect)
                mp_drawing.draw_landmarks(frame, handslms, mp_hands.HAND_CONNECTIONS)

        # Calculate realtive coordinates with hand landmarks 
        if landmarks:
            new_landmarks = calc_relative_coord(landmarks)
            
            # Preprocessing for input hand data
            input_hand = preprocess_handdata(new_landmarks.copy())

            # Input data for hand recognition
            pred_proba_hand = model_xgb.predict_proba(input_hand)
            # Predict hand input 
            pred_hand = np.argmax(pred_proba_hand)
            cv2.putText(frame, "Your gesture is number " + str(pred_hand), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    
    # To quit from application, press "q"
    if cv2.waitKey(1) == ord('q'): 
        break
        
    # Show the final output
    cv2.imshow("Output", frame)
    
# Release the webcam and destroy all active windows
video_capture.release()
cv2.destroyAllWindows()